# Converting PyTorch to TensorFlow Lite for xCORE Using ONNX

ONNX is an open format built to represent machine learning models. We can convert from PyTorch to ONNX, then from ONNX to TensorFlow, then from TensorFlow to TensorFlow Lite, and finally, run it through xformer to optimise it for xCORE.

In [1]:
!pip install torch
!pip install tensorflow
!pip install onnx
!pip install nvidia-pyindex
!pip install onnx-graphsurgeon
!pip install polygraphy
!pip install onnxruntime
!pip install onnxsim
!pip install simple_onnx_processing_tools
!pip install protobuf==3.20.3
!pip install h5py==3.7
!pip install onnx2tf
!pip install onnx-tf
!pip install tensorflow-probability

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Import PyTorch Model

For this example, we use mobilenet_v2.

In [2]:
import torch
import numpy as np

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

Using cache found in /Users/salmankhan/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

### Run inference on model (to test)

In [3]:
# Download an image to test against
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [4]:
# Download Image Labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-03-20 17:18:25--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.003s  

2023-03-20 17:18:26 (3.91 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [5]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [6]:
from PIL import Image
from torchvision import transforms

# Open testing image
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with torch.no_grad():
    output = model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Show top categories per image
vals, idxs = torch.topk(probabilities, 5)
pytorch_results = [(categories[idx], prob) for (idx, prob) in zip(idxs.tolist(), vals.tolist())]
print(pytorch_results)


[('Samoyed', 0.8303042650222778), ('Pomeranian', 0.06988772749900818), ('keeshond', 0.012964067049324512), ('collie', 0.010797766037285328), ('Great Pyrenees', 0.009886772371828556)]


## Convert to ONNX


In [7]:
batch_size = 1
channels = 3
height = 224
width = 224

sample_input = torch.rand((batch_size, channels, height, width))

onnx_model_path = "mobilenet_v2.onnx"

torch.onnx.export(
    model,
    sample_input,
    onnx_model_path,
    input_names=['image'],
    output_names=['probabilities']
)

### Check ONNX Output

In [8]:
import onnxruntime 

ort_session = onnxruntime.InferenceSession(onnx_model_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def softmax(xs):
    return np.exp(xs)/sum(np.exp(xs))

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_batch)}
ort_outs = ort_session.run(None, ort_inputs)

data = zip(range(len(ort_outs[0][0])), softmax(ort_outs[0][0]))

onnx_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
print(onnx_results)

[('Samoyed', 0.830305), ('Pomeranian', 0.06988746), ('keeshond', 0.012964004), ('collie', 0.010797733), ('Great Pyrenees', 0.009886744)]


## Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

#### Download & Extract Images from Data Set

In [9]:
!mkdir ./imagenet-dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz -O ./imagenet-dataset/imagenette2-320.tgz
!tar -xf ./imagenet-dataset/imagenette2-320.tgz -C ./imagenet-dataset/

mkdir: ./imagenet-dataset: File exists
--2023-03-20 17:19:01--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.62.192, 54.231.170.56, 52.217.46.102, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.62.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341663724 (326M) [application/x-tar]
Saving to: ‘./imagenet-dataset/imagenette2-320.tgz’

./imagenet-dataset/ 100%[===================>] 325.83M  1.45MB/s    in 3m 33s  

2023-03-20 17:22:35 (1.53 MB/s) - ‘./imagenet-dataset/imagenette2-320.tgz’ saved [341663724/341663724]



In [10]:
import os
import glob
import random

def representative_dataset():
    all_files = glob.glob(os.path.join("./imagenet-dataset/", '**/*.JPEG'), recursive=True)
    
    # Randomly select a subset of images
    sampled_files = random.sample(all_files, 100)
    
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Iterate over the sampled images and preprocess them
    for image_path in sampled_files:
        pil_image = Image.open(image_path).convert("RGB")
        pytorch_tensor = preprocess(pil_image).unsqueeze(0)
        img_array = pytorch_tensor.numpy()
        
        reshaped = np.swapaxes(img_array, 1, 3)
        yield [reshaped]

## Using onnx-tensorflow (no longer maintained) / Not Currently Working

Official ONNX package, however no longer officially maintained: https://github.com/onnx/onnx-tensorflow

In [11]:
# import onnx
# from onnx_tf.backend import prepare

# saved_model_path = "saved_model"
# onnx_model = onnx.load(onnx_model_path)
# prepare(onnx_model).export_graph(saved_model_path)

In [12]:
# import tensorflow as tf

# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8 
# converter.inference_output_type = tf.int8

# tflite_model = converter.convert()

# # Save the model.
# tflite_model_path = 'mobilenet_v2.tflite'
# with open(tflite_model_path, 'wb') as f:
#   f.write(tflite_model)

## Using onnx2tf

Using unofficial package: https://github.com/PINTO0309/onnx2tf

### Convert ONNX to Keras

In [13]:
import onnx2tf

keras_model = onnx2tf.convert(
    input_onnx_file_path=onnx_model_path,
    output_signaturedefs=True,
    non_verbose=True,
)

### Convert Keras to TFLite

In [14]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

# Save the model.
tflite_model_path = 'mobilenet_v2.tflite'
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


### Check it Worked

In [16]:
tf_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
tf_interpreter.allocate_tensors()

tf_input_details = tf_interpreter.get_input_details()
tf_output_details = tf_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
# (Pytorch model expects C x H x W but TF expects H x W x C)
tf_input_data = np.swapaxes(input_batch.detach().numpy(), 1, 3)

tf_interpreter.set_tensor(tf_input_details[0]['index'], tf_input_data)
tf_interpreter.invoke()

tf_output_data = tf_interpreter.get_tensor(tf_output_details[0]['index'])
data = zip(range(len(tf_output_data[0])), softmax(tf_output_data[0]))
tf_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]

print(tf_results)

[('Great Pyrenees', 0.25053665), ('Samoyed', 0.11746472), ('Old English sheepdog', 0.08676046), ('Siberian husky', 0.08676046), ('Border collie', 0.07456401)]


# Analysing Models

Defined below is a function to print out the operator counts of each model.

In [ ]:
import io
from contextlib import redirect_stdout

def get_operator_counts(model_content):
    with io.StringIO() as buf, redirect_stdout(buf):
        tf.lite.experimental.Analyzer.analyze(model_content=model_content)
        model_structure = buf.getvalue()

    operators = [op.strip().split(" ")[1].split("(")[0] for op in model_structure.split("\n") if "Op#" in op]
    op_counts = {}
    for operator in operators:
        if operator in op_counts:
            op_counts[operator] = op_counts[operator]+1
        else:
            op_counts[operator] = 1
        
    return (len(operators), op_counts)

def print_operator_counts(model_content):
    total_op_count, op_counts = get_operator_counts(model_content)
    print(f"{'Operator'.upper():<20} {'Count'.upper():>6}")
    print("-"*20 + " " + "-"*6)
    
    for operator, count in op_counts.items():
        print(f"{operator.lower():<20} {count:>6}")
        
    print("-"*20 + " " + "-"*6)
    print(f"{'Total'.upper():<20} {total_op_count:>6}")
    print("-"*20 + " " + "-"*6)

In [ ]:
print_operator_counts(tflite_model)